<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# NRPy+ LaTeX Parser Interface

## Author: Ken Sible

## The following module will demonstrate a recursive descent parser for LaTeX.

### NRPy+ Source Code for this module:
1. [latex_parser.py](../edit/latex_parser.py); [\[**tutorial**\]](Tutorial-LaTeX_Parser_Interface.ipynb) The latex_parser.py script will convert a LaTeX sentence to a SymPy expression using the following function: parse(sentence).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#intro): Lexical Analysis and Syntax Analysis
1. [Step 2](#demo): Parser Demonstration and Sandbox
1. [Step 3](#tensor): Tensor Support with Einstein Notation
1. [Step 4](#error): Exception Handling and Index Checking
1. [Step 5](#latex_pdf_output): $\LaTeX$ PDF Output

<a id='intro'></a>

# Step 1: Lexical Analysis and Syntax Analysis \[Back to [top](#toc)\]
$$\label{intro}$$

In the following section, we discuss [lexical analysis](https://en.wikipedia.org/wiki/Lexical_analysis) (lexing) and [syntax analysis](https://en.wikipedia.org/wiki/Parsing) (parsing). In the process of lexical analysis, a lexer will tokenize a character string, called a sentence, using substring pattern matching (or tokenizing). We implemented a regex-based lexer for NRPy+, which does pattern matching using a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) for each token pattern. In the process of syntax analysis, a parser will receive a token iterator from the lexer and build a parse tree containing all syntactic information of the language, as specified by a [formal grammar](https://en.wikipedia.org/wiki/Formal_grammar). We implemented a [recursive descent parser](https://en.wikipedia.org/wiki/Recursive_descent_parser) for NRPy+, which will build a parse tree in [preorder](https://en.wikipedia.org/wiki/Tree_traversal#Pre-order_(NLR)), starting from the root [nonterminal](https://en.wikipedia.org/wiki/Terminal_and_nonterminal_symbols), using a [right recursive](https://en.wikipedia.org/wiki/Left_recursion) grammar. The following right recursive, [context-free grammar](https://en.wikipedia.org/wiki/Context-free_grammar) was written for parsing [LaTeX](https://en.wikipedia.org/wiki/LaTeX), adhering to the canonical (extended) [BNF](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form) notation used for describing a context-free grammar.

```
<EXPRESSION>    -> <TERM> { ( '+' | '-' ) <TERM> }*
<TERM>          -> <FACTOR> { [ '/' ] <FACTOR> }*
<FACTOR>        -> <BASE> { '^' <EXPONENT> }*
<BASE>          -> [ '-' ] ( <ATOM> | <SUBEXPR> )
<EXPONENT>      -> <BASE> | '{' <BASE> '}' | '{' '{' <BASE> '}' '}'
<ATOM>          -> <COMMAND> | <OPERATOR> | <NUMBER> | <TENSOR>
<SUBEXPR>       -> '(' <EXPRESSION> ')' | '[' <EXPRESSION> ']' | '\' '{' <EXPRESSION> '\' '}'
<COMMAND>       -> <FUNC> | <FRAC> | <SQRT> | <NLOG> | <TRIG>
    ⋮            ⋮
```

<small>**Source**: Robert W. Sebesta. Concepts of Programming Languages. Pearson Education Limited, 2016.</small>

In [1]:
from latex_parser import *
import sympy as sp, sys

In [2]:
lexer = Lexer(); lexer.initialize(r'(1 + x/n)^n')
print(', '.join(token for token in lexer.tokenize()))

LPAREN, INTEGER, PLUS, LETTER, DIVIDE, LETTER, RPAREN, CARET, LETTER


In [3]:
expr = parse_expr(r'(1 + x/n)^n')
print(expr, '\n  >>', sp.srepr(expr))

(1 + x/n)**n 
  >> Pow(Add(Integer(1), Mul(Pow(Symbol('n', real=True), Integer(-1)), Symbol('x', real=True))), Symbol('n', real=True))


#### `Grammar Derivation: (1 + x/n)^n`
```
<EXPRESSION> -> <TERM>
             -> <FACTOR>
             -> <BASE>^<EXPONENT>
             -> <SUBEXPR>^<EXPONENT>
             -> (<EXPRESSION>)^<EXPONENT>
             -> (<TERM> + <TERM>)^<EXPONENT>
             -> (<FACTOR> + <TERM>)^<EXPONENT>
             -> (<BASE> + <TERM>)^<EXPONENT>
             -> (<ATOM> + <TERM>)^<EXPONENT>
             -> (<NUMBER> + <TERM>)^<EXPONENT>
             -> (<INTEGER> + <TERM>)^<EXPONENT>
             -> (1 + <TERM>)^<EXPONENT>
             -> (1 + <FACTOR> / <FACTOR>)^<EXPONENT>
             -> ...
```

<a id='demo'></a>

# Step 2: Parser Demonstration and Sandbox \[Back to [top](#toc)\]
$$\label{demo}$$

We implemented a wrapper function for the `parse()` method that will accept a LaTeX sentence and return a SymPy expression. Furthermore, the entire parsing module was designed for extendibility. We apply the following procedure for extending parser functionality to include an unsupported LaTeX command: append that command to the grammar dictionary in the Lexer class with the mapping regex:token, write a grammar abstraction (similar to a regular expression) for that command, add the associated nonterminal (the command name) to the command abstraction in the Parser class, and finally implement the straightforward (private) method for parsing the grammar abstraction. We shall demonstrate the extension procedure using the `\sqrt` LaTeX command.

```<SQRT> -> <SQRT_CMD> [ '[' <INTEGER> ']' ] '{' <EXPRESSION> '}'```
```
def _sqrt(self):
    self.expect('SQRT_CMD')
    if self.accept('LBRACK'):
        integer = self.lexer.lexeme
        self.expect('INTEGER')
        root = Rational(1, integer)
        self.expect('RBRACK')
    else: root = Rational(1, 2)
    self.expect('LBRACE')
    expr = self._expression()
    self.expect('RBRACE')
    if root == Rational(1, 2):
        return sqrt(expr)
    return Pow(expr, root)
```

In addition to expression parsing, we included support for equation parsing, which will produce a dictionary mapping LHS $\mapsto$ RHS, where LHS must be a symbol, and insert that mapping into the global namespace of the previous stack frame, as demonstrated below.

$$ \mathit{s_n} = \left(1 + \frac{1}{n}\right)^n $$

In [4]:
parse(r'\text{s_n} = \left(1 + \frac{1}{n}\right)^n')
print('s_n =', s_n)

s_n = (1 + 1/n)**n


We implemented robust error messaging using the custom `ParseError` exception, which should handle every conceivable case to identify, as detailed as possible, invalid syntax inside of a LaTeX sentence. The following are some runnable examples of possible error messages.

In [5]:
Parser.ignore_override(); Parser.continue_parsing = False

In [6]:
try: parse_expr(r'x^{{2}} + 2$')
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: x^{{2}} + 2$
                       ^
unexpected '$' at position 11


In [7]:
try: parse_expr(r'\sqrt[0.1]{2x^2}')
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: \sqrt[0.1]{2x^2}
                  ^
expected token INTEGER at position 6


In [8]:
try: parse_expr(r'\int_0^5 2x^2dx')
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: \int_0^5 2x^2dx
            ^
unsupported command '\int' at position 0


In the sandbox code cell below, you can experiment with the LaTeX parser using the wrapper function `parse(sentence)`, where sentence must be a [raw string](https://docs.python.org/3/reference/lexical_analysis.html) to interpret a backslash as a literal character rather than an [escape sequence](https://en.wikipedia.org/wiki/Escape_sequence).

In [9]:
# Write Sandbox Code Here

<a id='tensor'></a>

# Step 3: Tensor Support with Einstein Notation \[Back to [top](#toc)\]
$$\label{tensor}$$

In the following section, we demonstrate the current parser support for tensor notation using the Einstein summation convention. The first example will parse an equation for a tensor contraction, the second will parse an equation for raising an index using the metric tensor, and the third will parse an align enviroment with an equation dependency. In each example, every tensor should appear either on the LHS of an equation or inside of a configuration before appearing on the RHS of an equation. Moreover, the parser will raise an exception upon violation of the Einstein summation convention, i.e. an invalid free or bound index.

**Configuration Grammar**

```
<MACRO>     -> <PARSE> | <SREPL> | <VARDEF> | <KEYDEF> | <ASSIGN> | <IGNORE>
<PARSE>     -> <PARSE_MACRO> <ASSIGNMENT> { ',' <ASSIGNMENT> }*
<SREPL>     -> <SREPL_MACRO> <STRING> <ARROW> <STRING> { ',' <STRING> <ARROW> <STRING> }*
<VARDEF>    -> <VARDEF_MACRO> { '-' <OPTION> }* <VARIABLE> { ',' <VARIABLE> }* [ '(' <DIMENSION> ')' ]
<KEYDEF>    -> <KEYDEF_MACRO> <BASIS_KWRD> <BASIS> | <INDEX_KWRD> <INDEX>
<ASSIGN>    -> <ASSIGN_MACRO> { '-' <OPTION> }* <VARIABLE> { ',' <VARIABLE> }*
<IGNORE>    -> <IGNORE_MACRO> <STRING> { ',' <STRING> }*
<OPTION>    -> <DRV_TYPE> [ <PRIORITY> ] | <SYMMETRY> | <WEIGHT> '=' <NUMBER>
<BASIS>     -> <BASIS_KWRD> '{' <LETTER> { ',' <LETTER> }* '}'
<INDEX>     -> ( <LETTER> | '[' <LETTER> '-' <LETTER> ']' ) '(' <DIMENSION> ')'
```

#### Example 1. [Tensor Contraction](https://en.wikipedia.org/wiki/Tensor_contraction)

In [10]:
output = parse(r"""
    % vardef -nosym 'hUD' (4D)
    h = h^\mu{}_\mu
""", verbose=True)
print(output); display(output)

(Tensor(hUD, 4D), Scalar(h))


(Tensor(hUD, 4D), Scalar(h))

In [11]:
print('h =', h)

h = hUD00 + hUD11 + hUD22 + hUD33


#### Example 2. [Tensor Index Raising](https://en.wikipedia.org/wiki/Raising_and_lowering_indices) ([Metric Tensor](https://en.wikipedia.org/wiki/Metric_tensor))

In [12]:
output = parse(r"""
    % vardef -metric 'gUU' (3D)
    % vardef 'vD' (3D)
    v^\mu = g^{\mu\nu} v_\nu
""")
print(output); display(output)

('gUU', 'epsilonDDD', 'gdet', 'gDD', 'vD', 'vU')


('gUU', 'epsilonDDD', 'gdet', 'gDD', 'vD', 'vU')

In [13]:
print('vU =', vU)

vU = [gUU00*vD0 + gUU01*vD1 + gUU02*vD2, gUU01*vD0 + gUU11*vD1 + gUU12*vD2, gUU02*vD0 + gUU12*vD1 + gUU22*vD2]


#### Example 3. [Permutation Symbol](https://en.wikipedia.org/wiki/Levi-Civita_symbol) ([Cross Product](https://en.wikipedia.org/wiki/Cross_product))

In [14]:
output = parse(r"""
    % vardef 'epsilonDDD' (3D)
    % vardef 'vU' (3D), 'wU' (3D)
    u_i = \epsilon_{ijk} v^j w^k
""")
print(output); display(output)

('epsilonDDD', 'vU', 'wU', 'uD')


('epsilonDDD', 'vU', 'wU', 'uD')

In [15]:
print('uD =', uD)

uD = [vU1*wU2 - vU2*wU1, -vU0*wU2 + vU2*wU0, vU0*wU1 - vU1*wU0]


#### Example 4 (1). [Covariant Derivative](https://en.wikipedia.org/wiki/Covariant_derivative) ([Four Current](https://en.wikipedia.org/wiki/Four-current))

The following are contextually inferred, dynamically generated, and injected into the global namespace for expansion of the covariant derivative $\nabla_\nu F^{\mu\nu}$
$$
\begin{align*}
    \Gamma^\mu_{ba} &= \frac{1}{2} g^{\mu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \Gamma^\nu_{ba} &= \frac{1}{2} g^{\nu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \nabla_a F^{\mu \nu} &= \partial_a F^{\mu \nu} + \Gamma^\mu_{b a} F^{b \nu} + \Gamma^\nu_{b a} F^{\mu b}
\end{align*}
$$

In [16]:
output = parse(r"""
    % vardef -numeric -anti01 'FUU' (4D)
    % vardef -numeric -metric 'gDD' (4D)
    % vardef -const 'k'
    J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
""")
print(output); display(output)

('FUU', 'gDD', 'epsilonUUUU', 'gdet', 'gUU', 'FUU_dD', 'gDD_dD', 'GammaUDD', 'FUU_cdD', 'JU')


('FUU',
 'gDD',
 'epsilonUUUU',
 'gdet',
 'gUU',
 'FUU_dD',
 'gDD_dD',
 'GammaUDD',
 'FUU_cdD',
 'JU')

#### Example 4 (2). Diacritic Support

In [17]:
output = parse(r"""
    % vardef -anti01 'FUU' (4D)
    % vardef -metric 'ghatDD' (4D)
    % vardef -const 'k'
    J^\mu = (4\pi k)^{-1}
        \vphantom{numeric} \hat{\nabla}_\nu F^{\mu\nu}
""")
print(output); display(output)

('FUU', 'ghatDD', 'epsilonUUUU', 'ghatdet', 'ghatUU', 'FUU_dD', 'ghatDD_dD', 'GammahatUDD', 'FUU_cdhatD', 'JU')


('FUU',
 'ghatDD',
 'epsilonUUUU',
 'ghatdet',
 'ghatUU',
 'FUU_dD',
 'ghatDD_dD',
 'GammahatUDD',
 'FUU_cdhatD',
 'JU')

#### Example 5 (1). [Schwarzschild Metric](https://en.wikipedia.org/wiki/Schwarzschild_metric)

In [18]:
output = parse(r"""
    % vardef 'deltaDD' (4D)
    % vardef -const 'G', 'M'
    % parse g_{\mu\nu} = \delta_{\mu\nu}
    \begin{align}
        g_{0 0} &= -\left(1 - \frac{2GM}{r}\right) \\
        g_{1 1} &=  \left(1 - \frac{2GM}{r}\right)^{-1} \\
        g_{2 2} &= r^{{2}} \\
        g_{3 3} &= r^{{2}} \sin^2\theta
    \end{align}
    % assign -metric 'gDD'
""")
print(output); display(output)
Parser.continue_parsing = True

('deltaDD', 'gDD', 'epsilonUUUU', 'gdet', 'gUU')


('deltaDD', 'gDD', 'epsilonUUUU', 'gdet', 'gUU')

In [19]:
display(sp.Matrix(gDD))

Matrix([
[2*G*M/r - 1,                0,    0,                  0],
[          0, 1/(-2*G*M/r + 1),    0,                  0],
[          0,                0, r**2,                  0],
[          0,                0,    0, r**2*sin(theta)**2]])

#### Example 5 (2). [Kretschmann Scalar](https://en.wikipedia.org/wiki/Kretschmann_scalar) ([Einstein Tensor](https://en.wikipedia.org/wiki/Einstein_tensor))

In [20]:
output = parse(r"""
    % keydef basis [t, r, \theta, \phi]
    \begin{align}
        R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu}
            - \partial_\nu \Gamma^\alpha_{\beta\mu}
            + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu}
            - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
        R^{\alpha\beta\mu\nu} &= g^{\beta a} g^{\mu b} g^{\nu c} R^\alpha{}_{a b c} \\
        R_{\alpha\beta\mu\nu} &= g_{\alpha a} R^a{}_{\beta\mu\nu} \\
        K &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu} \\
        R_{\beta\nu} &= R^\alpha{}_{\beta\alpha\nu} \\
        R &= g^{\beta\nu} R_{\beta\nu} \\
        G_{\beta\nu} &= R_{\beta\nu} - \frac{1}{2}g_{\beta\nu}R
    \end{align}
""")
print(output); display(output)

('GammaUDD', 'RUDDD', 'RUUUU', 'RDDDD', 'K', 'RDD', 'R', 'GDD')


('GammaUDD', 'RUDDD', 'RUUUU', 'RDDDD', 'K', 'RDD', 'R', 'GDD')

In [21]:
sp.simplify(sp.Matrix(RDD))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [22]:
display(sp.Matrix(GammaUDD[0][:][:]))

Matrix([
[                        0, -G*M/(r**2*(2*G*M/r - 1)), 0, 0],
[-G*M/(r**2*(2*G*M/r - 1)),                         0, 0, 0],
[                        0,                         0, 0, 0],
[                        0,                         0, 0, 0]])

In [23]:
display(sp.Matrix(GammaUDD[1][:][:]))

Matrix([
[G*M*(-2*G*M/r + 1)/r**2,                          0,                 0,                               0],
[                      0, -G*M/(r**2*(-2*G*M/r + 1)),                 0,                               0],
[                      0,                          0, -r*(-2*G*M/r + 1),                               0],
[                      0,                          0,                 0, -r*(-2*G*M/r + 1)*sin(theta)**2]])

In [24]:
display(sp.Matrix(GammaUDD[2][:][:]))

Matrix([
[0,   0,   0,                      0],
[0,   0, 1/r,                      0],
[0, 1/r,   0,                      0],
[0,   0,   0, -sin(theta)*cos(theta)]])

In [25]:
display(sp.Matrix(GammaUDD[3][:][:]))

Matrix([
[0,   0,                     0,                     0],
[0,   0,                     0,                   1/r],
[0,   0,                     0, cos(theta)/sin(theta)],
[0, 1/r, cos(theta)/sin(theta),                     0]])

For the Schwarzschild metric, the Kretschmann scalar $K$ has the property that $K\to\infty$ as $r\to 0$, and hence the metric and spacetime itself are undefined at the point of infinite curvature $r=0$, indicating the prescence of a physical singularity since the Kretschmann scalar is an [invariant quantity](https://en.wikipedia.org/wiki/Curvature_invariant_(general_relativity)) in general relativity.

In [26]:
display(sp.simplify(K))

48*G**2*M**2/r**6

In a [vacuum region](https://en.wikipedia.org/wiki/Vacuum_solution_(general_relativity)#:~:text=In%20general%20relativity%2C%20a%20vacuum,non%2Dgravitational%20fields%20are%20present.), such as the spacetime described by the Schwarzschild metric, $T_{\mu\nu}=0$ and hence $G_{\mu\nu}=0$ since $G_{\mu\nu}=8\pi G\,T_{\mu\nu}$ ([Einstein Equations](https://en.wikipedia.org/wiki/Einstein_field_equations)).

In [27]:
display(sp.simplify(sp.Matrix(GDD)))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

#### Example 6 (1). [Extrinsic Curvature](https://en.wikipedia.org/wiki/Curvature) ([ADM Formalism](https://en.wikipedia.org/wiki/ADM_formalism))

In [28]:
output = parse(r"""
    % keydef basis [r, \theta, \phi]
    \begin{align}
        \gamma_{ij} &= g_{ij} \\
        % assign -metric 'gammaDD'
        \beta_i &= g_{0 i} \\
        \alpha &= \sqrt{\gamma^{ij}\beta_i\beta_j - g_{0 0}} \\
        K_{ij} &= \frac{1}{2\alpha}\left(\nabla_i \beta_j + \nabla_j \beta_i\right) \\
        K &= \gamma^{ij} K_{ij}
    \end{align}
""")
print(output); display(output)

('gammaDD', 'epsilonUUU', 'gammadet', 'gammaUU', 'betaD', 'alpha', 'betaD_cdD', 'KDD')


('gammaDD',
 'epsilonUUU',
 'gammadet',
 'gammaUU',
 'betaD',
 'alpha',
 'betaD_cdD',
 'KDD')

For the Schwarzschild metric (defined in the previous example), the extrinsic curvature in the ADM formalism should evaluate to zero.

In [29]:
display(sp.Matrix(KDD))

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

#### Example 6 (2). [Hamiltonian/Momentum Constraint](https://en.wikipedia.org/wiki/Hamiltonian_constraint)

In [30]:
output = parse(r"""
    \begin{align}
        K^{ij} &= \gamma^{ik} \gamma^{jl} K_{kl} \\
        R_{ij} &= \partial_k \Gamma^k_{ij} - \partial_j \Gamma^k_{ik}
            + \Gamma^k_{ij}\Gamma^l_{kl} - \Gamma^l_{ik}\Gamma^k_{lj} \\
        R &= \gamma^{ij} R_{ij} \\
        E &= \frac{1}{16\pi}\left(R + K^{{2}} - K_{ij}K^{ij}\right) \\
        p_i &= \frac{1}{8\pi}\left(D_j \gamma^{jk} K_{ki} - D_i K\right)
    \end{align}
""")
Parser.continue_parsing = False
print(output); display(output)

('KUU', 'E', 'gammaUU_cdD', 'K_cdD', 'pD')


('KUU', 'E', 'gammaUU_cdD', 'K_cdD', 'pD')

Every solution to the Einstein Equations, including Schwarzschild, must satisfy the Hamiltonian constraint ($E=0$) and the Momentum constraint ($p_i=0$).

In [31]:
print('E = %s, pD = %s' % (sp.simplify(E), pD))

E = 0, pD = [0, 0, 0]


<a id='error'></a>

# Step 4: Exception Handling and Index Checking \[Back to [top](#toc)\]
$$\label{error}$$

We extended our robust error messaging using the custom `TensorError` exception, which should handle any inconsistent tensor dimension and any violation of the Einstein summation convention, specifically that a bound index must appear exactly once as a superscript and exactly once as a subscript in any single term and that a free index must appear in every term with the same position and cannot be summed over in any term.

In [32]:
try:
    parse(r"""
        % vardef 'TUD' (4D), 'uD' (4D)
        v^\mu = T^\mu_\nu u_\nu
    """)
except TensorError as err:
    print('TensorError: %s' % err)

TensorError: illegal bound index


In [33]:
try:
    parse(r"""
        % vardef 'TUD' (4D), 'uD' (4D)
        v^\mu = T^\mu_\nu u_\mu
    """)
except TensorError as err:
    print('TensorError: %s' % err)

TensorError: unbalanced free index


In [34]:
try:
    parse(r"""
        % vardef 'TUD' (4D), 'uD' (3D)
        v^\mu = T^\mu_\nu u_\mu
    """)
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: inconsistent dimension for index 'mu'


In [35]:
try:
    parse(r"""
        % vardef 'vD' (4D)
        T_{\mu\nu} = v_\mu w_\nu
    """)
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: T_{\mu\nu} = v_\mu w_\nu
                               ^
cannot index undefined tensor 'wD' at position 46


In [36]:
try:
    parse(r"""
        % vardef 'vD' (2D)
        v_0 = x^{{2}} + 2x \\
        v_1 = y\sqrt{x} \\
        T_{\mu\nu} = \partial_\nu v_\mu
    """)
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: cannot differentiate symbolically without specifying a basis


In [37]:
try:
    parse(r"""
        R^\alpha{}_{\beta\mu\nu} = \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu}
            + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu}
    """)
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: R^\alpha_{\beta\mu\nu} = \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu}
                                                  ^
cannot generate christoffel symbol without defined metric 'g'


In [38]:
try:
    parse(r"""
        % vardef -anti01 'FUU' (4D)
        % vardef -const 'k'
        J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
    """)
except ParseError as err:
    print('ParseError: %s' % err)

ParseError: J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
                                  ^
cannot generate covariant derivative without defined metric 'g'


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-LaTeX_Parser_Interface.pdf](Tutorial-LaTeX_Parser_Interface.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [39]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-LaTeX_Parser_Interface")

Created Tutorial-LaTeX_Parser_Interface.tex, and compiled LaTeX file to PDF
    file Tutorial-LaTeX_Parser_Interface.pdf
